In [6]:
import pandas as pd
import numpy as np
from plotnine import *
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
time = "2024-12-31_13:33"
time = "2025-01-02_08:33"
time = "2025-01-03_06:00"
time = "2025-01-04_09:44"
time = "2025-01-06_17:52"
time = "2025-01-20_17:15"
time = "2025-01-20_18:26"
# time = "2025-01-21_11:28"


raw_df = pd.read_csv(f"/projects/genomic-ml/da2343/ml_project_2/unsupervised/kmeans/results/{time}_results.csv")
raw_df

In [ ]:
def analyze_performance(raw_df):
    # Group by relevant parameters and calculate averages
    group_cols = ['instrument', 'num_clusters', 'clustering_algorithm', 'train_period', 
                 'reverse_test', 'random_seed']
    
    # Calculate mean performance metrics for each parameter combination
    results_df = raw_df.groupby(group_cols).agg({
        'test_profit_factor': 'mean',
        'test_sharpe_ratio': 'mean',
        'test_win_ratio': 'mean',
        'test_num_trades': 'mean'
    }).reset_index()
    
    # Calculate combined score
    results_df['combined_score'] = (
        results_df['test_profit_factor'] * 0.4 +
        results_df['test_sharpe_ratio'] * 0.4 +
        (results_df['test_num_trades'] / results_df['test_num_trades'].max()) * 0.2
    )
    
    # Create display DataFrame with proper formatting
    display_df = pd.DataFrame({
        'Instrument': results_df['instrument'],
        'Clusters': results_df['num_clusters'],
        'Algorithm': results_df['clustering_algorithm'],
        'Train Period': results_df['train_period'].astype(str) + 'w',
        'Rev.Test': results_df['reverse_test'],
        'Seed': results_df['random_seed'],
        'PF': results_df['test_profit_factor'].round(3),
        'Sharpe': results_df['test_sharpe_ratio'].round(3),
        'Win%': (results_df['test_win_ratio'] * 100).round(1),
        'Trades': results_df['test_num_trades'].round(1),
        'Score': results_df['combined_score'].round(3)
    })
    
    # Sort by instrument name
    display_df = display_df.sort_values('Instrument')
    
    # Apply styling with color gradients
    styled_df = display_df.style\
        .format({
            'PF': '{:.3f}',
            'Sharpe': '{:.3f}',
            'Win%': '{:.1f}%',
            'Trades': '{:.1f}',
            'Score': '{:.3f}'
        })\
        .background_gradient(subset=['Score'], cmap='YlOrRd')\
        .background_gradient(subset=['PF'], cmap='YlOrRd')\
        .background_gradient(subset=['Sharpe'], cmap='YlOrRd')\
        .background_gradient(subset=['Win%'], cmap='YlOrRd')\
        .background_gradient(subset=['Trades'], cmap='YlOrRd')\
        .set_properties(**{
            'text-align': 'right',
            'font-family': 'monospace',
            'padding': '5px'
        })\
        .hide(axis="index")
    
    return results_df, styled_df

# Example usage:
results_df, styled_df = analyze_performance(raw_df)
styled_df  # Display the styled table

In [ ]:
results_df